In [ ]:
#default_exp stats

In [ ]:
#hide
%load_ext autoreload
%autoreload 2            #Reload the code automatically
%config Completer.use_jedi = False

In [ ]:
#hide 
import sys, os
from pathlib import Path

# Insert in Path Project Directory
sys.path.insert(0, str(Path().cwd().parent))

In [ ]:
#export
from typing import Iterable
import pandas as pd
import numpy as np
from fastcore.foundation import L, listify
from rich.console import Console
from rich.theme import Theme
from rich.progress import Progress
from rfpye.utils import get_files
from rfpye.parser import parse_bin
from loguru import logger

ImportError: cannot import name 'console' from 'rfpye.constants' (g:\Meu Drive\repos\Code\rfpye\rfpye\constants.py)

In [ ]:
custom_theme = Theme({"info": "dim cyan", "warning": "magenta", "danger": "bold red"})
console = Console(theme=custom_theme)

In [ ]:
#exporti
# For scripts
config = {
    "handlers": [
        {"sink": "stats.log", "serialize": True, 'rotation': "1 month", 'compression' :'zip', 'backtrace': True, 'diagnose': True},
    ],
}
logger.configure(**config)

## Resumo Estatístico
A seguinte função recebe um DataFrame cujas linhas são as diferentes varreduras do espectro, cada uma com seu timestamp, e colunas as diferentes frequências centrais medidas. Essa função é chamada pela função `extract_bin_stats`

In [ ]:
#export
@logger.catch
def filter_spectrum(
    df: pd.DataFrame,
    time_start: str = None,
    time_stop: str = None,
    freq_start: str = None,
    freq_stop: str = None,
) -> pd.DataFrame:
    """Recebe o arquivo de espectro df e retorna de acordo com os filtros

    Args:
        df (pd.DataFrame): Arquivo de espectro. Timestamp como linhas e frequências como colunas
        time_start (str): Timestamp de início. Se None filtra desde o início do arquivo
        time_stop (str): Timestamp de fim. Se None filtra até o fim do arquivo
        freq_start (str): Filtro inicial de frequência. Se None retorna desde a menor frequências
        freq_stop (str): Filtro Final de frequência. Se None retorna até a maior frequência.

    Returns:
        pd.DataFrame: DataFrame com Frequência, min, max e mean após os filtros aplicados.
    """
    df = df.copy()
    if time_start is None:
        time_start = "01/01/2000"
    if time_stop is None:
        time_stop = "31/12/2100"
    try:
        time_start = pd.to_datetime(time_start)
        time_stop = pd.to_datetime(time_stop)
    except pd.errors.ParserError:
        logger.error(
            f"[bold red blink] Datas inválidas! Verifique as strings de data {freq_start} e {freq_stop}"
        )

    try:
        df.set_index("index", inplace=True)
        df.index = pd.to_datetime(df.index)
    except pd.errors.KeyError:
        if not isinstance(df.index, pd.DatetimeIndex):
            logger.warning(
                f"Não foi passado uma coluna ou índice com datetime a ser filtrado, todas as linhas serão processadas",
                exc_info=True,
            )
            time_start = 0
            time_stop = df.shape[0]

    cols = df.columns.values.astype("float")
    rows = df.index.values

    if freq_start is None:
        freq_start = 0
    if freq_stop is None:
        freq_stop = np.inf

    filtered_cols = df.columns[(float(freq_start) <= cols) & (cols <= float(freq_stop))]
    filtered_rows = df.index[(time_start <= rows) & (rows <= time_stop)]
    if len(filtered_cols) == 0 or len(filtered_rows) == 0:
        return None
    count = filtered_rows.shape[0]
    array = df.loc[filtered_rows, filtered_cols].values
    freq = filtered_cols.values.astype("float32")
    min_ = array.min(axis=0)
    max_ = array.max(axis=0)
    mean = array.mean(axis=0)
    return pd.DataFrame(
        {"Frequency": freq, "Min": min_, "Max": max_, "Mean": mean, "Count": count}
    )

# Extração, Estatísticas e Salvamento dos Arquivos de
> Nesse módulo temos algumas funções que extraem estatísticas dados determinados filtros

In [ ]:
#exporti
def read_meta(filename):
    ext = filename.suffix
    if ext == ".csv":
        df = pd.read_csv(filename)
    elif ext == ".xlsx":
        df = pd.read_excel(filename, engine="openpyxl")
    elif ext == ".fth":
        df = pd.read_feather(filename)
        if "wallclock_datetime" in df.columns:
            df.set_index("wallclock_datetime", inplace=True)
    else:
        raise ValueError(f"Extension {ext} not implemented")
    return df

## Processamento e Extração 
A função a seguir é um wrapper de toda funcionalidade desta biblioteca. Ela recebe o caminho `entrada` para um arquivo `.bin` ou pasta contendo vários arquivos `.bin`, extrai os metadados e os dados de espectro. Mescla o timestamp dos metadados com o arquivo de espectro e salva ambos na pasta `saida`. Essa pasta é usada como repositório e cache dos dados processados que serão utilizados pela função `extract_bin_stats`.

In [ ]:
#export
def process_bin(
    entrada: str,
    saida: str,
    recursivo: bool = False,
    pastas: Iterable[str] = None,
    levels: bool = False,
    substituir: bool = False,
    dtype: str = "float16",
) -> None:
    """Recebe uma pasta ou arquivo bin, processa e salva os metadados e espectro na saida.

    Args:
        entrada (str): Caminho para a Pasta ou Arquivo .bin
        saida (str): Pasta onde salvar os arquivos processados
        recursivo (bool, optional): Buscar os arquivos de entrada recursivamente. Defaults to False.
        pastas (Iterable[str], optional): Limitar a busca a essas pastas. Defaults to None.
        levels (bool, optional): Extrair e salvar os dados de espectro. Defaults to False.
        substituir (bool, optional): Reprocessar arquivos já processados?. Defaults to False.
        dtype (str, optional): Tipo de dados a salvar o espectro. Defaults to "float16".
    """

    entrada = Path(entrada)
    if entrada.is_file():
        lista_bins = [entrada]
    else:
        lista_bins = get_files(
            entrada, extensions=[".bin"], recurse=recursivo, folders=pastas
        )
    parsed_bins = {}
    meta_path = Path(f"{saida}/meta")
    levels_path = Path(f"{saida}/levels")
    meta_path.mkdir(exist_ok=True, parents=True)
    levels_path.mkdir(exist_ok=True, parents=True)
    log_meta = Path(f"{saida}/log_meta.txt")
    log_levels = Path(f"{saida}/log_levels.txt")
    if substituir:
        done_meta = set()
        done_levels = set()
    else:

        done_meta = (
            set(log_meta.read_text().split("\n")) if log_meta.exists() else set()
        )
        done_levels = (
            set(log_levels.read_text().split("\n")) if log_levels.exists() else set()
        )

    console.rule("Lista de Arquivos a serem processados", style="bold red")
    console.print(
        [f.name for f in lista_bins],
        style="bold white",
        overflow="fold",
        justify="left",
    )
    if not lista_bins:
        console.print(":sleeping: Nenhum arquivo .bin a processar :zzz:")
        return

    if not levels:
        lista_bins = [f for f in lista_bins if f.name not in done_meta]
    else:
        lista_bins = [f for f in lista_bins if f.name not in done_levels]

    if not lista_bins:
        console.print(":sleeping: Nenhum arquivo novo a processar :zzz:")
        console.print(
            ":point_up: use --substituir no terminal ou substituir=True na chamada caso queira reprocessar os bins e sobrepôr os arquivos existentes :wink:"
        )
        return

    try:

        with Progress(transient=True, auto_refresh=False) as progress:
            bins = progress.track(
                lista_bins,
                total=len(lista_bins),
                description="[green]Processando Blocos Binários",
            )

            for file in bins:
                progress.console.print(f"[cyan]Processando Blocos de: [red]{file.name}")
                parsed_bins[file.name] = parse_bin(file)
                progress.refresh()

            lista_meta = [(k, v) for k, v in parsed_bins.items() if k not in done_meta]

            if lista_meta:
                blocks = progress.track(
                    lista_meta,
                    total=len(lista_meta),
                    description="[cyan]Exportando Metadados",
                )
                for filename, block_dict in blocks:
                    progress.console.print(f"[cyan]Extraindo Metadados de: [red]{file}")
                    export_metadata(filename, block_dict, meta_path, ext=".fth")
                    done_meta.add(file)
                    progress.refresh()
            if levels:
                lista_levels = lista_meta = [
                    (k, v) for k, v in parsed_bins.items() if k not in done_levels
                ]
                if lista_levels:
                    bins = progress.track(
                        lista_levels,
                        total=len(lista_levels),
                        description="[grey]Exportando Dados de Espectro",
                    )
                    for file, block_obj in bins:
                        progress.console.print(
                            f"[grey]Extraindo Espectro de: [red]{file}"
                        )
                        meta_index = []
                        blocks = block_obj["blocks"]
                        for (tipo, tid) in blocks.keys():
                            if tipo not in SPECTRAL_BLOCKS:
                                continue
                            meta_file = Path(
                                f"{meta_path}/{file}-B_{tipo}_TId_{tid}.fth"
                            )
                            if not meta_file.exists():
                                export_meta(
                                    file,
                                    block_obj,
                                    meta_path,
                                    ext=".fth",
                                )
                                done_meta.add(file)
                            meta_df = read_meta(meta_file)
                            meta_index.append(meta_df.index.tolist())
                        export_level(
                            file,
                            block_obj,
                            levels_path,
                            ext=".fth",
                            index=meta_index,
                            dtype=dtype,
                        )
                        done_levels.add(file)
                        progress.refresh()
        console.print("kbô :satisfied:")
    finally:
        log_meta.write_text("\n".join(sorted(list(done_meta))))
        log_levels.write_text("\n".join(sorted(list(done_levels))))

A função a seguir é a que será mais comumente chamada por outro módulo que utilizar esta lib. Ela recebe o caminho para um arquivo `.bin` e retorna um DataFrame com Frequência, Máximo, Mínimo e Média dos dados de Espectro presentes no arquivo `.bin`

In [ ]:
#export
def extract_bin_stats(
    filename: str,
    time_start: str = None,
    time_stop: str = None,
    freq_start: str = None,
    freq_stop: str = None,
    cache: str = CACHE_FOLDER,
) -> pd.DataFrame:
    """Recebe o caminho para um arquivo CRFS bin e retorna um dataframe com o resumo estatístico dos dados de espectro

    Args:
        filename (str): Caminho para o arquivo bin
        time_start (str): Timestamp de início. Se None filtra desde o início do arquivo
        time_stop (str): Timestamp de fim. Se None filtra até o fim do arquivo
        freq_start (str): Filtro inicial de frequência. Se None retorna desde a menor frequências
        freq_stop (str): Filtro Final de frequência. Se None retorna até a maior frequência.
        cache (str, optional): Caminho para a pasta de cache. Default é criar uma pasta oculta .cache no diretório atual.

    Returns:
        pd.DataFrame: Dataframe contendo o resumo estatístico do arquivo
    """

    cache = Path(cache)
    cache.mkdir(exist_ok=True, parents=True)
    filename = Path(filename)
    if filename.is_dir():
        filenames = get_files(filename, extensions=[".bin"])
    else:
        filenames = listify(filename)

    cached_files = get_files(cache / "levels")
    files = L()
    for filename in filenames:
        while True:
            # TODO filter based on metadata
            subset = cached_files.filter(lambda name: filename.stem in str(name))
            if not len(subset):
                process_bin(entrada=filename, saida=cache, levels=True)
            else:
                break
        files += subset
        subset = L()

    dfs = files.map(pd.read_feather)
    tids = files.map(lambda x: x.stem.split("_")[-1])
    spectra = dfs.map(
        filter_spectrum,
        time_start=time_start,
        time_stop=time_stop,
        freq_start=freq_start,
        freq_stop=freq_stop,
    )
    spectra = [(i, s) for i, s in zip(tids, spectra) if s is not None]
    columns = ["Tid", "Frequency", "Min", "Max", "Mean"]
    out = pd.DataFrame(columns=columns)
    if not spectra:
        log.warning(
            f"Os parâmetros repassados não correspondem a nenhum dado espectral do arquivo",
            exc_info=True,
        )
        return out
    for i, df in spectra:
        df["Tid"] = i
    spectra = [s for i, s in spectra]
    spectra = pd.concat(spectra)
    if len(spectra.Frequency) == len(spectra.Frequency.unique()):
        return spectra[columns]
    gb = spectra.groupby(["Tid", "Frequency"])
    out = gb.apply(appended_mean)
    Min = gb.min()["Min"]
    Max = gb.max()["Max"]
    Mean = gb.apply(appended_mean)
    out = pd.concat([Min, Max, Mean], axis=1).reset_index()
    out.columns = columns
    return out

Chamada da função somente fornecendo o caminho do arquivo `.bin`

Pela saída do código acima, vemos que o arquivo `.bin` foi processado, seus metadados e espectro extraídos e salvos. Como não passamos uma pasta de saída uma pasta local `.cache` é criada e os arquivos são salvos nela. Posteriormente o arquivo de espectro no cache é lido e o resumo estatístico das frequências no tempo é retornado.

Se chamarmos novamente a função com os mesmos argumentos, dessa vez a execução será mais rápida por conta do cache e assim o arquivo `.bin` não precisa ser processado novamente.

In [ ]:
dados = extract_bin_stats(binfile) ; dados

NameError: name 'extract_bin_stats' is not defined

In [ ]:
#exporti
def appended_mean(df: pd.Series) -> float:
    """Recebe um agrupamento do DataFrame e retorna sua média ponderada pela coluna Count

    Args:
        df (pd.DataFrame): Groupby do DataFrame

    Returns:
        float: Média Ponderada da linha pela coluna Count
    """
    return (df["Count"] * df["Mean"]).sum() / df["Count"].sum()

Vemos que o arquivo possui frequências de 70MHz a 110MHz. Se tivermos interessados em faixas menos, podemos filtrá-las. Por exemplo, vamos filtrar pela faixa de FM somente `88 a 108`:

In [ ]:
dados = extract_bin_stats(binfile, freq_start=88, freq_stop=108) ; dados

NameError: name 'extract_bin_stats' is not defined

Para filtrarmos os dados estatísticos relativo a um tempo específico, precisamos saber de antemão qual o período específico o arquivo `.bin` compreende, se passarmos um período de tempo inválido, é retornado um DataFrame vazio e uma mensagem de aviso é salva no log.

In [ ]:
dados = extract_bin_stats(binfile, time_start='2021-05-21') ; dados

NameError: name 'extract_bin_stats' is not defined

In [ ]:
dados = extract_bin_stats(binfile, time_stop='2020-05-12') ; dados

NameError: name 'extract_bin_stats' is not defined

Esse arquivo específico compreende o período de `Timestamp('2020-12-01 15:34:21.578869') a Timestamp('2020-12-01 16:13:53.920250')`, um período de menos de uma hora.

Basta passarmos uma string de data válida, as horas, minutos e segundos são opcionais.

In [ ]:
dados = extract_bin_stats(binfile, time_start='2020-12-01 16:00') ; dados

NameError: name 'extract_bin_stats' is not defined

In [ ]:
dados = extract_bin_stats(binfile, time_start='01/12/2020 16:00') ; dados

NameError: name 'extract_bin_stats' is not defined

In [ ]:
dados = extract_bin_stats(binfile, time_stop='2020-12-01 16:00') ; dados

NameError: name 'extract_bin_stats' is not defined

Se quisermos filtrar para constar somente a faixa de FM e somente os 15 minutos de 15:45 a 16:00 do dia 01/12/2020 

In [ ]:
dados = extract_bin_stats(binfile, 
                          time_start='01/12/2020 15:45',
                          time_stop='2020-12-01 16:00',
                          freq_start=88,
                          freq_stop=108) 
dados

NameError: name 'extract_bin_stats' is not defined